In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()


messages=[
    { 'role': 'user', 'content': "Сгенерируй код сортировки пузырьком на Python"}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
#outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
#print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._py

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  Н


Invalid input. Must be one of ('y', 'yes', '1', 'n', 'no', '0', '')


Add token as git credential? (Y/n)  Y


Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /trinity/home/team16/workspace/.cache/huggingface/token
Login successful


In [3]:
model.push_to_hub("my-controlnet-model", variant="fp16")

SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5", trust_remote_code=True)
tokenizer.Save

In [ ]:
model

In [ ]:
#model.model.layers = model.model.layers[:-4]

In [ ]:
model

In [ ]:
import numpy as np 
print(f'num params before SVD compression = {sum( [ np.prod(item.size())  for item in model.parameters() ])}')

In [1]:
import sys
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM

import torch
class LaserLinear(torch.nn.Module):

    def __init__(self, weight: torch.Tensor, ratio: float):
        super().__init__()
        max_rank = min(weight.shape)
        q = int(max_rank * ratio)
        U, sigma, V = torch.svd_lowrank(weight.float(), q=q, niter=2)
        
        self.U = torch.nn.Parameter(U.contiguous())#.type(weight.dtype)
        self.sigma = torch.nn.Parameter(sigma.contiguous())#.type(weight.dtype)
        self.V = torch.nn.Parameter(V.contiguous())#.type(weight.dtype)

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return input @ (self.U @ torch.diag(self.sigma) @ self.V.T).T

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
AutoModelForCausalLM.from_pretrained('coder-7b-laser-19-23-01', local_files_only = True)

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at coder-7b-laser-19-23-01 were not used when initializing LlamaForCausalLM: ['model.layers.19.mlp.gate_proj.sigma', 'model.layers.23.self_attn.o_proj.V', 'model.layers.21.self_attn.q_proj.sigma', 'model.layers.21.self_attn.v_proj.sigma', 'model.layers.23.mlp.gate_proj.sigma', 'model.layers.22.self_attn.k_proj.U', 'model.layers.22.mlp.down_proj.U', 'model.layers.19.self_attn.o_proj.sigma', 'model.layers.19.mlp.gate_proj.V', 'model.layers.20.self_attn.q_proj.U', 'model.layers.20.mlp.gate_proj.sigma', 'model.layers.20.self_attn.o_proj.sigma', 'model.layers.21.mlp.up_proj.V', 'model.layers.22.mlp.down_proj.sigma', 'model.layers.22.mlp.gate_proj.sigma', 'model.layers.22.self_attn.o_proj.U', 'model.layers.22.mlp.gate_proj.V', 'model.layers.20.mlp.gate_proj.U', 'model.layers.22.self_attn.v_proj.U', 'model.layers.19.self_attn.k_proj.sigma', 'model.layers.23.self_attn.q_proj.sigma', 'model.layers.19.mlp.up_proj.U', 'model.layers.19.self_attn.o_proj.V', 'mod

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(102400, 4096)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm

In [ ]:
mlp_names = [
    "mlp.down_proj",
    "mlp.up_proj",
    "mlp.gate_proj",
    "self_attn.q_proj",
    "self_attn.k_proj",
    "self_attn.v_proj",
    "self_attn.o_proj",
]
sys.stdout.flush()

ratio = 0.1
treshold = 5
curr_compressed = 0

for i, layer_index in enumerate( reversed(range(len(model.model.layers))) ):
    if curr_compressed < treshold and i > 5: 
        for mlp_name in mlp_names:
            original_linear = eval(f"model.model.layers[layer_index].{mlp_name}")
            weight = original_linear.weight
            #print(f'weight type = {weight.dtype}')
            exec(f"model.model.layers[layer_index].{mlp_name"} = LaserLinear(weight, ratio).type(torch.bfloat16)")
            sys.stdout.flush()
        curr_compressed += 1
    #torch.save(model.state_dict(), f"mistral-7b-instruct-laser-{ratio}")

In [ ]:
torch.save({'model': model, 'tokenizer': tokenizer},"laser-19-23-01.pt")

/tmp/ipykernel_336895/3189298871.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load('laser-19-23-01.pt')['model']
/trinity/home/team16/workspace/.local/lib/pytho

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(102400, 4096)
    (layers): ModuleList(
      (0-18): 19 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
      (19-23): 5 x LlamaDecoderLayer(
    

In [8]:
model

NameError: name 'model' is not defined

In [9]:
import numpy as np 
print(f'num params after SVD compression = {sum( [ np.prod(item.size())  for item in model.parameters() ])}')

num params after SVD compression = 6058177771


In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [7]:
from vllm import LLM, SamplingParams
llm = LLM(model='./coder-7b-laser-19-23-01',tokenizer = 'deepseek-ai/deepseek-coder-7b-instruct-v1.5',  enforce_eager=True,max_model_len = 512,  max_num_batched_tokens = 512)
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens = 512)

INFO 08-10 17:35:34 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='./coder-7b-laser-19-23-01', speculative_config=None, tokenizer='deepseek-ai/deepseek-coder-7b-instruct-v1.5', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=512, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=./coder-7b-laser-19-23-01, use_v2_block_manager=False, enable_prefix_caching=False)


/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 08-10 17:35:35 model_runner.py:720] Starting to load model ./coder-7b-laser-19-23-01...


[W810 17:35:35.412931278 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [gn35.zhores]:55737 (errno: 97 - Address family not supported by protocol).


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


KeyError: 'model.layers.19.mlp.down_proj.U'

In [ ]:
tokenizer = torch.load('laser-19-23-01.pt')['model']

In [11]:
HUMAN_EVAL ="../human-eval/data/HumanEval.jsonl.gz"

from typing import Dict, Iterable

def read_problems(evalset_file: str = HUMAN_EVAL) -> Dict[str, Dict]:
    return {task["task_id"]: task for task in stream_jsonl(evalset_file)}


def stream_jsonl(filename: str) -> Iterable[Dict]:
    """
    Parses each jsonl line and yields it as a dictionary
    """
    if filename.endswith(".gz"):
        with open(filename, "rb") as gzfp:
            with gzip.open(gzfp, 'rt') as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        yield json.loads(line)
    else:
        with open(filename, "r") as fp:
            for line in fp:
                if any(not x.isspace() for x in line):
                    yield json.loads(line)


def write_jsonl(filename: str, data: Iterable[Dict], append: bool = False):
    """
    Writes an iterable of dictionaries to jsonl
    """
    if append:
        mode = 'ab'
    else:
        mode = 'wb'
    filename = os.path.expanduser(filename)
    if filename.endswith(".gz"):
        with open(filename, mode) as fp:
            with gzip.GzipFile(fileobj=fp, mode='wb') as gzfp:
                for x in data:
                    gzfp.write((json.dumps(x) + "\n").encode('utf-8'))
    else:
        with open(filename, mode) as fp:
            for x in data:
                fp.write((json.dumps(x) + "\n").encode('utf-8'))

def generate_one_completion(prompt, max_new_tokens):
    prompt = f"You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer\n### Instruction:\n{prompt}. Return only code without explanation.\n### Response:\n"
    outputs = llm.generate(prompt, sampling_params)
    # Print the outputs.
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
    return out

In [12]:
model.device

device(type='cuda', index=0)

In [13]:
import gzip, json 
from tqdm import tqdm
problems = read_problems()
MAX_NEW_TOKENS = 128
NUM_SAMPLES_PER_TASK = 1



samples = [
    dict(task_id=task_id, completion=generate_one_completion(problems[task_id]["prompt"], MAX_NEW_TOKENS))
    for task_id in tqdm(problems)
    for _ in range(NUM_SAMPLES_PER_TASK)
]

  0%|          | 0/164 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100015 for open-end generation.
  1%|          | 1/164 [00:04<12:56,  4.76s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100015 for open-end generation.
  1%|          | 2/164 [00:09<12:47,  4.74s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100015 for open-end generation.
  2%|▏         | 3/164 [00:14<12:41,  4.73s/it]The attention mask and the pad token id were not set. As a conse

In [14]:
write_jsonl('coder-laser-25-30-01.json', samples)

SyntaxError: invalid syntax (1610238083.py, line 7)

In [16]:
arr = [3, 7, 1, 6]
bubble_sort(arr)

NameError: name 'bubble_sort' is not defined

In [15]:
messages=[
    { 'role': 'user', 'content': "Сгенерируй код сортировки пузырьком на Python. В ответ выдай только код программы"}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100015 for open-end generation.


Sure, here is simple Python code for sorting a list using bubble sort algorithm:
```python
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        swapped = False
        for j in range(n - i - 1):
            if arr[j > arr[j+1]:
                arr[j], arr[j+]
                swapped = True
        if not swapped:
            break
    return arr
```
This code works as follows:
1. Loop through the array from the first element to the last element (i=0 to n).
2. For each element, compare it with next element (i+1). If current element is greater than next element, swap them.
3. After each iteration, smallest element will be at its correct position. So, next iteration will be from next element to last element.
4. Repeat steps until no swaps are needed indicating that array is sorted.

Note: This code assumes that array elements are comparable. If array contains non-comparable elements, this code will raise TypeError. Also, this code modifies original array, if you want to k

In [11]:
1 / 6.7

0.14925373134328357